In [2]:
import os
import pandas as pd
import requests

#os.chdir('/Users/thomas/Documents/reveals/CLIENTS/MASSENA/Massena_PS/services/web/input')

#### chargement des produits 

In [3]:
#ticker = pd.read_excel('./produitStructure.xlsx', sheet_name="TICKER")
#ssjacents = pd.read_excel('./produitStructure.xlsx', sheet_name="SSJACENT")
produit_structure = pd.read_excel('./produitStructure.xlsx', sheet_name="Autocall")

In [4]:
columns_renamed = {
    'Code ISIN': 'isin',
    'Ticker du sous-jacent': 'ticker',
    'Devise': 'devise',
    'Intitulé commercial': 'shortName',
    'typeCommentaire': 'description',
    'Montant Souscrit': 'investissement',
    'Type de barrière': 'barriere',
    'Type': 'produit',
    'Emetteur': 'emetteur',
    'Maturité': 'maturite',
    'Type de produit': 'produit2',
    'clientele_cible': 'clientele',
    'Remuneration': 'remuneration',
    'typePanier': 'panier',

    'Date credit en compte': 'dateAchat',
    'Date d\'émission': 'dateEmission',
    'Date d\'échéance': 'dateEchFinale',
    'Date de remboursement effectif': 'dateRembEff',

    #'': 'periodeRemb',
    'Periode': 'periodeRemb',
    'Période de non Call': 'periodeNocall'
}

produit_structure.rename(columns=columns_renamed, inplace=True)

for col in columns_renamed.values():
    if col not in produit_structure.columns:
        produit_structure[col] = None
    

PS_final = produit_structure[columns_renamed.values()]
PS_final = PS_final.fillna('')
PS_final['dateAchat'] = PS_final['dateAchat'].dt.strftime('%d/%m/%Y').astype(str).replace('nan', '01/01/1900')
PS_final['dateEchFinale'] = PS_final['dateEchFinale'].dt.strftime('%d/%m/%Y').astype(str).replace('nan', '01/01/1900')
PS_final['dateEmission'] = PS_final['dateEmission'].dt.strftime('%d/%m/%Y').astype(str).replace('nan', '01/01/1900')
PS_final['dateRembEff'] = PS_final['dateRembEff'].dt.strftime('%d/%m/%Y').astype(str).replace('nan', '01/01/1900')

#PS_final["periodeRemb"] = PS_final["periodeRemb"].replace('','0').astype(int)
PS_final["maturite"] = PS_final["maturite"].replace('','0').astype(int)
PS_final["investissement"] = PS_final["investissement"].replace('','0').astype(int)
PS_final["maturite"] = PS_final["maturite"].replace('','0').astype(int)

# From the column description, remove the html tags
PS_final['description'] = PS_final['description'].str.replace('<[^<]+?>', '')
# From the column description, remvoe characters that are between _
PS_final['description'] = PS_final['description'].str.replace('_.*_', '')




/var/folders/my/rs8pgdv168n0_4d_v__1kp0h0000gn/T/ipykernel_6232/810293565.py:47: FutureWarning: The default value of regex will change from True to False in a future version.
  PS_final['description'] = PS_final['description'].str.replace('<[^<]+?>', '')
/var/folders/my/rs8pgdv168n0_4d_v__1kp0h0000gn/T/ipykernel_6232/810293565.py:49: FutureWarning: The default value of regex will change from True to False in a future version.
  PS_final['description'] = PS_final['description'].str.replace('_.*_', '')


In [5]:
def normParam(x):
    return x.strip().upper() if len(x)>0 else 'NONE'

for col in ["emetteur","produit","produit2","clientele","remuneration","panier","devise","barriere","periodeRemb"]:
    PS_final[col]=PS_final[col].apply(normParam)


In [6]:
json_input = PS_final.to_json(orient='records')

# api-endpoint
URL = "http://127.0.0.1:5001/api/setproducts"

r = requests.post(url = URL, json = json_input)
print(r)

<Response [200]>


---

#### Chargement des sous-jacents : 

In [214]:
# Set columns name as row with index 581 and set the previous column name as a row
ssjacents = pd.read_excel('./produitStructure.xlsx', sheet_name="SSJACENT")
ssjacents.columns = ssjacents.loc[581].reset_index(drop=True)
ssjacents.head()
ssjacents.shape

(838, 3)

In [215]:
ticker = pd.read_excel('./produitStructure.xlsx', sheet_name="TICKER")
columns_renamed = {'CODE ISIN': 'isin','TICKER': 'ticker','Sous-jacent': 'shortName','Prix': 'price','DatePrix': 'priceDate',
}
                   
ticker.rename(columns=columns_renamed, inplace=True)
                   
INSTRUMENT_final = ticker[columns_renamed.values()]
INSTRUMENT_final = INSTRUMENT_final.fillna('')

# Transform priceDate to DateTime
INSTRUMENT_final['priceDate'] = pd.to_datetime(INSTRUMENT_final['priceDate'], format='%d/%m/%Y', errors='coerce')
INSTRUMENT_final['priceDate'] = INSTRUMENT_final['priceDate'].dt.strftime('%Y-%m-%d').astype(str).replace('nan', '01/01/1900')
INSTRUMENT_final["price"] = INSTRUMENT_final["price"].replace('','0').astype(float)

In [216]:
ssjacents = pd.read_excel('./produitStructure.xlsx', sheet_name="SSJACENT")
ssjacents.columns = ssjacents.loc[581].reset_index(drop=True)
ssjacents.columns.name = None
ssjacents['ISIN'] = ssjacents['ISIN'].str.replace('[^a-zA-Z0-9]', '')
ssjacents = ssjacents.rename(columns={'ISIN': 'isin','NOM': 'shortName'})
ssjacents_reworked = pd.merge(INSTRUMENT_final, ssjacents, on='shortName', how='inner')
ssjacents_reworked[["shortName", "ticker", "isin_x", "isin_y"]]

ssjacents_reworked = ssjacents_reworked.rename(columns={'isin_x':'idInstrument','isin_y':'idProduit'})
ssjacents_reworked=ssjacents_reworked[['idProduit','ticker','price','priceDate','strike']]


/Users/david/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


In [217]:
ssjacents_reworked=ssjacents_reworked.rename(columns={'idProduit':'isinProduit','ticker':'TickerSj'})
ssjacents_reworked

,isinProduit,TickerSj,price,priceDate,strike
0,XS2349887278,1919 HK Equity,8.77,01/01/1900,15.3539
1,XS2278659623,2330 TT Equity,387.00,01/01/1900,570
2,XS2349887278,2603 TT Equity,137.50,01/01/1900,136
3,CH0567768038,700 HK Equity,206.20,01/01/1900,584
4,CH0569863316,AAPL UW Equity,149.45,01/01/1900,116.97
...,...,...,...,...,...
799,XS2114132710,XXX,0.00,01/01/1900,NaN
800,XS2123780970,XXX,0.00,01/01/1900,NaN
801,XS2238705623,XXX,0.00,01/01/1900,NaN
802,XS2335545963,XXX,0.00,01/01/1900,NaN


In [218]:
json_input = ssjacents_reworked.to_json(orient='records')
# api-endpoint
URL = "http://127.0.0.1:5001/api/setssjacents"

r = requests.post(url = URL, json = json_input)
print(r)

<Response [200]>


---

#### chargement des échéances :

In [219]:
dfEch=pd.read_excel('./produitStructure.xlsx', sheet_name="BARRIERE")
dfEch.head()

,Periode,Pourcentage,ISIN,noncall
0,1,1.000,CH0283710108,OUI
1,2,1.000,CH0283710108,NON
2,3,0.975,CH0283710108,NON
3,4,0.950,CH0283710108,NON
4,5,0.925,CH0283710108,NON


In [220]:
dfEch=dfEch.rename(columns={'ISIN':'isinProduit','Pourcentage':'Percent','Periode':'Period'})
dfEch['noncall']=dfEch['noncall'].apply(lambda x: True if x=='OUI'else False)
dfEch['type']='Remb'

dfEch.head()

,Period,Percent,isinProduit,noncall,type
0,1,1.000,CH0283710108,True,Remb
1,2,1.000,CH0283710108,False,Remb
2,3,0.975,CH0283710108,False,Remb
3,4,0.950,CH0283710108,False,Remb
4,5,0.925,CH0283710108,False,Remb


In [221]:
json_input = dfEch.to_json(orient='records')


In [222]:
json_input = dfEch.to_json(orient='records')
# api-endpoint
URL = "http://127.0.0.1:5001/api/setecheances"

r = requests.post(url = URL, json = json_input)
print(r)

<Response [200]>


In [17]:
portfolios = pd.read_csv('./BANKVISTA_portfolios_20230503_20230504-190004.csv', sep=';')
securities = pd.read_csv('./BANKVISTA_securities_20230504_20230505-080502.csv', sep=';')
positions = pd.read_csv('./BANKVISTA_positions_20230503_20230504-190003.csv', sep=';')

In [32]:
positions[~pd.isna(positions['ISIN'])].columns

Index(['PortfolioId', 'Date', 'Security', 'ISIN', 'Bloomberg', 'SecurityCcy',
       'Quantity', 'Price', 'ValueSecurityCcy', 'ValueAccountCcy',
       'AccountCcy', 'AccruedInterest'],
      dtype='object')

In [33]:
portfolios[portfolios['AccountID']=="0000180491Q"].columns

Index(['Status', 'EvaluationCurrency', 'AccountName', 'AccountID', 'GroupID',
       'AccountList', 'CodeName', 'OtherID', 'ManagementType', 'CustodyBank',
       'InsuranceCompany', 'InsuranceType', 'InvestmentPolicy',
       'ManagementGroup', 'ClientNature', 'AccountManager1', 'AccountManager2',
       'RelationshipManager', 'CreationDate', 'OpeningDate', 'ClosingDate'],
      dtype='object')

In [67]:
merge_df=pd.merge(positions[['PortfolioId','ISIN']], portfolios[['AccountID','RelationshipManager']], right_on='AccountID', left_on='PortfolioId', how='inner')

# Get unique list of ISIN in PS_final table
isin_list=PS_final['isin'].tolist()


managers = merge_df[merge_df['ISIN'].isin(isin_list)]['RelationshipManager'].unique()

# For each manager, create a new table with name, surname, mail and role
managers_df = pd.DataFrame(columns=['name', 'surname', 'mail', 'role', 'password'])
for manager in managers:
    role = 'FRONT'
    if manager == 'MARTIN LIONEL':
        role = 'ADMIN'
    managers_df = managers_df.append({'name': manager.split(' ')[0], 'surname': manager.split(' ')[1].title(), 'mail': manager.split(' ')[0].lower() + '.' + manager.split(' ')[1] + '@massena.lu', 'role': role, 'password': 'massena'}, ignore_index=True)

managers_df

/var/folders/my/rs8pgdv168n0_4d_v__1kp0h0000gn/T/ipykernel_6232/3974815577.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  managers_df = managers_df.append({'name': manager.split(' ')[0], 'surname': manager.split(' ')[1].title(), 'mail': manager.split(' ')[0] + '.' + manager.split(' ')[1] + '@massena.lu', 'role': role, 'password': 'massena'}, ignore_index=True)
/var/folders/my/rs8pgdv168n0_4d_v__1kp0h0000gn/T/ipykernel_6232/3974815577.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  managers_df = managers_df.append({'name': manager.split(' ')[0], 'surname': manager.split(' ')[1].title(), 'mail': manager.split(' ')[0] + '.' + manager.split(' ')[1] + '@massena.lu', 'role': role, 'password': 'massena'}, ignore_index=True)
/var/folders/my/rs8pgdv168n0_4d_v__1kp0h0000gn/T/ipykernel_6232/3974815577.py:16: Fu

,name,surname,mail,role,password
0,SCHON,Pascale,SCHON.PASCALE@massena.lu,FRONT,massena
1,NGUYEN,Minh-Tri,NGUYEN.MINH-TRI@massena.lu,FRONT,massena
2,BERRIH,Abdel,BERRIH.ABDEL@massena.lu,FRONT,massena
3,MARTIN,Lionel,MARTIN.LIONEL@massena.lu,ADMIN,massena
4,CUILLIERE,Sandra,CUILLIERE.SANDRA@massena.lu,FRONT,massena


In [52]:
managers

array(['SCHON PASCALE', 'NGUYEN MINH-TRI', 'BERRIH ABDEL',
       'MARTIN LIONEL', 'CUILLIERE SANDRA'], dtype=object)

In [65]:
manager.split(' ')[0]
#Set from manager.split(' ')[0] to upper case only for first letter
manager.split(' ')[0].title()

'Cuilliere'

In [5]:
def newEcheances():
    dfEch=pd.read_excel('./produitStructure.xlsx', sheet_name="BARRIERE")

    dfEch=dfEch.rename(columns={'ISIN':'isinProduit','Pourcentage':'Percent','Periode':'Period'})
    dfEch['noncall']=dfEch['noncall'].apply(lambda x: True if x=='OUI'else False)
    dfEch['type']='Remb'

    json_input = dfEch.to_json(orient='records')

    json_input = dfEch.to_json(orient='records')
    
    # api-endpoint
    URL = "http://127.0.0.1:5001/api/setecheances"

    r = requests.post(url = URL, json = json_input)
    print(r)

newEcheances()


<Response [200]>
